# How to use `contaminante`

This tutorial will show you how to use `contaminante`. If you'd like to try it yourself, you can use `contaminante` online, in the cloud, for free! Click [here](https://colab.research.google.com/github/christinahedges/contaminante/blob/master/tutorials/Colaboratory-Notebook.ipynb) to run `contaminante` online using Google's Colaboratory. 

## Using `contaminante` on *Kepler* data

To demonstrate `contaminante` we'll first use *Kepler* data. First we'll need a target to try `contaminante` on. I've chosen *KIC 6804648*. This target was observed during the prime *Kepler* mission, and was flagged as a planet candidate. In fact, the target has a contaminating eclipsing binary. This binary is only obvious in some of the *Kepler* quarters. 

Below we run the target through `contaminante`. Running this cell should take less than 5 minutes.

In [1]:
import contaminante

In [2]:
fig, result = contaminante.calculate_contamination(targetid='KIC {}'.format(6804648),
                               period=0.700606,
                               t0=131.59767,
                               duration=0.993/24,
                               mission='kepler')

TypeError: calculate_contamination() missing 1 required positional argument: 'tpfs'

Using `contaminante` we can see two pieces of evidence that this target is contaminated.
1. There is a significant offset between the center of the **target** (green cross) in the image, and the **source of the transiting signal** (red cross).
2. There is a significant difference between the **target** phase curve (green phase curve) and the **source of the transiting signal** phase curve (red phase curve).

The result dictionary contains the depth and positions of the target and the "contamintor", including errors. It also contains a flag for whether the target is "contaminated". The user is encouraged to 1) look at the phase curves 2) look at the positions and transit depths before claiming that a target is contaminated.

In [ ]:
result

To compare, we can look at a target that is a true, confirmed planet. Below I run the parameters for **Kepler-10** through `contaminate`.

In [ ]:
fig, result = contaminante.calculate_contamination(targetid='KIC {}'.format(11904151),
                               period=0.837491,
                               t0=2454964.57513 - 2454833,
                               duration=1.8076/24,
                               mission='kepler')

Sometimes there will be no significant transiting source that was not the target, and so there will be no red cross in the image, and no red phase curve in the phase curve diagram. Sometimes there will be a weak detection that there are other pixels that contain the transit, but there is frequently no significant shift if

1. The two sources line up in the image
2. There is no significant difference between the target aperture and the source aperture.

Cases such as this can suggest the aperture you are using may not be optimal to recover all of the transiting signal.

In [ ]:
result

## Using `contaminante` on *TESS* Data

`contaminante` works on TESS data too. The background scattered light is removed using principle component analysis. For targets that are available in the TESS pipeline TPF products, the TPFs will be used. If no TPF is available, the data will be cut out of the FFI's using the TESSCut API from MAST.

In [ ]:
fig, result = contaminante.calculate_contamination(targetid="TIC 267263253",
                        period=4.12688,
                        t0=2458325.78297 - 2457000,
                        duration=0.3, mission='tess', bin_points=100)

In [ ]:
result

## Using `contaminante` on *K2* Data

`contaminante` works on K2 data too. The motion noise is removed using the same Self Flat Fielding technique used in `lightkurve`. Because of the K2 motion the results may be a little harder to interpret. For example, below there is a slight shift in the centroid, but the light curve from that target is not different from the main target. This is likely due to the pipeline apertures for K2 being slightly too small.

In [ ]:
fig, result = contaminante.calculate_contamination(targetid="EPIC 211732801",
                        period=2.1316925,
                        t0=2308.407161,
                        duration=0.3, mission='K2', bin_points=5)

In [ ]:
result

## Usage notes

1. **Different quarters, campaigns and sectors.** If a target has multiple quarters, campaigns or sectors you can expect each dataset to have some slight offset, due to the target falling on different pixels.

2. **Shallower contaminator light curves.** `contaminante` looks at each pixel individually to see if there is a significant transit signal. Because faint pixels can contribute a transiting signal at a lower, less significant level, some faint pixels can be missed in contaminante. In the case that the contaminator light curve is **shallower** than the target light curve, it is likely that some faint pixels have been missed from the optimum aperture. This does not indicate that there is any contamination.

